## Calculating The Distance between Houses and Schools Using Haversine Formula



![globe](https://user-images.githubusercontent.com/67468718/116970776-6a506b00-ac6d-11eb-80a6-2922f6f28929.JPG)

## Contents<a id='Contents'></a>
 * [1. Introduction](#1_Introduction)
 * [2. Objective](#2_Objective)
 * [3. Sourcing and Loading](#3_Sourcing_and_Loading)
    * [3.1 Import relevant libraries](#3.1_Import_relevant_libraries)
    * [3.2 Load and view the data](#3.2_Load_and_view_the_data)
 * [4. Haversine formula](#4_Haversine_formula)
    * [4.1 Prepare the data](#4.1_Prepare_the_data)
    * [4.2 Define Haversine formula Function](#4.2_Define_Haversine_formula_Function)
 * [5. Haversine formula output Modification](#5_Haversine_formula_output_Modification)
    * [5.1 Melting the output](#5.1_Melting_the_output)
    * [5.2 Merge the Dataframes](#5.2_Merge_the_Dataframes)
    * [5.3 Groupby the final dataframe](#5.3_Groupby_the_final_dataframe)
 * [6. Saving our final Dataframe](#6_Saving_our_final_Dataframe)


## 1. Introduction<a id='1_Introduction'></a>

**The Haversine formula** is perhaps the first equation to consider when understanding how to calculate distances on a sphere. The word "Haversine" comes from the function:

$haversine(θ) = sin²(θ/2)$

The following equation where φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km) is how we translate the above formula to include latitude and longitude coordinates. Note that angles need to be in radians to pass to trig functions:

$a = sin²(φB — φA/2) + cos φA * cos φB * sin²(λB — λA/2)$

$c = 2 * atan2( √a, √(1−a) )$

$d = R ⋅ c$

## 2. Objective<a id='2_Objective'></a>

**The Haversine formula** will help us to find all schools within 3 miles of every house and then we can take calculate the average GreatSchool Ratings for all schools within 3 miles of every house in our database.

## 3. Sourcing and Loading<a id='3_Sourcing_and_Loading'></a> 

### 3.1 Import relevant libraries<a id='3.1_Import_relevant_libraries'></a> 

In [1]:
# Install the necessary libraries:
import pandas as pd
import numpy as np
import haversine as hs
import os 

### 3.2 Load and view the data<a id='3.2_Load_and_view_the_data'></a>

In [ ]:
# Check Current Directory:
os.getcwd()

In [3]:
# Changing the directory:
os.chdir(r"path/Feature_Engineering/datasets")

In [ ]:
#Confirm the directory:
os.getcwd()

In [ ]:
# List files/folders in the cd:
os.listdir()

In [6]:
house_loc = pd.read_csv('house_loc.csv') # has all houses with long and lat
school_loc = pd.read_csv('school_loc.csv') # has all schools with long and lat
school_gsrating = pd.read_csv('GreatSchools_NCA_df.csv') # has all schools rating with long and lat

In [7]:
#let's review house data:
house_loc.head()

,address,lat,lon
0,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736
1,"270 birch creek dr, pleasanton, ca 94566",37.663585,-121.864563
2,"5324 brookside ct, pleasanton, ca 94588",37.689374,-121.922526
3,"4385 clovewood ln, pleasanton, ca 94588",37.680132,-121.915110
4,"4273 trolan ln, dublin, ca 94568",37.737748,-121.876525


In [8]:
house_loc.shape

(6002, 3)

In [9]:
school_loc.head()

,ncesId,lat,lon
0,63513005942,37.867493,-122.03093
1,63513005956,37.839924,-122.02790
2,63513005958,37.857260,-122.01954
3,60285000217,37.985570,-121.81191
4,60285011276,37.956287,-121.79193


## 4. Haversine formula<a id='4_Haversine_formula'></a>

### 4.1 Prepare the data<a id='4.1_Prepare_the_data'></a>

In [10]:
# concatenating lat and long to create a consolidated location as accepted by haversine function:
house_loc['coor'] = list(zip(house_loc.lat, house_loc.lon))
school_loc['coor'] = list(zip(school_loc.lat, school_loc.lon))

In [11]:
# let's review house data:
house_loc.head()

,address,lat,lon,coor
0,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,"(37.733978, -121.872736)"
1,"270 birch creek dr, pleasanton, ca 94566",37.663585,-121.864563,"(37.663585, -121.86456299999999)"
2,"5324 brookside ct, pleasanton, ca 94588",37.689374,-121.922526,"(37.689374, -121.922526)"
3,"4385 clovewood ln, pleasanton, ca 94588",37.680132,-121.915110,"(37.6801319, -121.9151098)"
4,"4273 trolan ln, dublin, ca 94568",37.737748,-121.876525,"(37.737747999999996, -121.8765253)"


In [12]:
# let's review school data:
school_loc.head()

,ncesId,lat,lon,coor
0,63513005942,37.867493,-122.03093,"(37.867492999999996, -122.03093)"
1,63513005956,37.839924,-122.02790,"(37.839923999999996, -122.0279)"
2,63513005958,37.857260,-122.01954,"(37.85726, -122.01953999999999)"
3,60285000217,37.985570,-121.81191,"(37.98557, -121.81191000000001)"
4,60285011276,37.956287,-121.79193,"(37.956286999999996, -121.79193000000001)"


In [13]:
#let'ws confrim shcool count:
school_loc['ncesId'].nunique()

234

In [14]:
#let's get schools list:
school_loc['ncesId'].unique()

array([63513005942, 63513005956, 63513005958, 60285000217, 60285011276,
       60285000209, 60285000211, 60285007492, 60285000156, 60285000215,
       60285009703, 60285000216, 60285008454, 60285011275, 60285010752,
       60285012278, 60285000207, 60285000208, 60285004440, 60285005724,
       60285000212, 60285000214, 60285000218, 62637009406, 62637003972,
       62637003936, 62637003969, 60780007661, 60780006909, 60780000746,
       60780000752, 60780000741, 60780000742, 60780000749, 60780009525,
       61674008855, 62637003958, 62637003983, 62637003934, 62637003942,
       62637003945, 62637003951, 62637003965, 62637003973, 62637003981,
       62637003985, 62637003944, 62637003957, 62637003962, 62637003984,
       62637003986, 62637003953, 63513005944, 63513012242, 63513005949,
       63513005947, 63513005948, 63513005951, 63513005952, 63513005957,
       63513000289, 63513007082, 63513005960, 63513008460, 63513005950,
       60001912078, 60001906929, 60001909273, 60001909275, 60001

In [15]:
#Now, let's define a list with all schools using school's ncesId: schools
schools = [63513005942, 63513005956, 63513005958, 60285000217, 60285011276,
       60285000209, 60285000211, 60285007492, 60285000156, 60285000215,
       60285009703, 60285000216, 60285008454, 60285011275, 60285010752,
       60285012278, 60285000207, 60285000208, 60285004440, 60285005724,
       60285000212, 60285000214, 60285000218, 62637009406, 62637003972,
       62637003936, 62637003969, 60780007661, 60780006909, 60780000746,
       60780000752, 60780000741, 60780000742, 60780000749, 60780009525,
       61674008855, 62637003958, 62637003983, 62637003934, 62637003942,
       62637003945, 62637003951, 62637003965, 62637003973, 62637003981,
       62637003985, 62637003944, 62637003957, 62637003962, 62637003984,
       62637003986, 62637003953, 63513005944, 63513012242, 63513005949,
       63513005947, 63513005948, 63513005951, 63513005952, 63513005957,
       63513000289, 63513007082, 63513005960, 63513008460, 63513005950,
       60001912078, 60001906929, 60001909273, 60001909275, 60001909276,
       60001909278, 60001908443, 60001913169, 60001912927, 60001909274,
       60001913862, 61674002123, 61674002104, 61674002106, 61674002118,
       61674002109, 61674002121, 61674002110, 61674002111, 61674005622,
       61674002107, 61674002120, 61674009847, 61674002132, 61674002136,
       63471005847, 63471005855, 63471011567, 61674002108, 61674002113,
       61674002114, 61674002115, 61674002116, 61674002124, 61674002126,
       61674002130, 61674002135, 62691004071, 62211002622, 62211002632,
       62211002635, 62211002620, 62211002618, 62211000052, 62211002623,
       62211002626, 62211002627, 62211002630, 62211002631, 62211002636,
       62211002628, 62211002624, 62211008438, 62211014134, 62403003615,
       62403000178, 62403003614, 62403003618, 62403009579, 62637003950,
       62403003612, 63060004751, 63060004750, 63060009995, 63060004753,
       63060004743, 63060004746, 63060004747, 63060004748, 63060004749,
       63060004752, 62637010756, 63060012268, 63060013016, 60002009286,
       60002009289, 60002009284, 60002009288, 60002008445, 60002009285,
       60002009282, 60002009283, 60002009279, 60002006951, 60002009682,
       60002009287, 60002009290, 60002008444, 62637003974, 62637003941,
       62637002956, 62637003967, 62637007319, 62637008916, 62637003977,
       62637003946, 62637003979, 63255005015, 63255005042, 63255005047,
       63255007081, 63255005030, 63255005032, 63255005037, 63255005040,
       63255005050, 63255005055, 63255005061, 63255005067, 63255005065,
       63543006044, 63543006053, 63543008984, 61029001132, 63543006033,
       63543006034, 63543006036, 63543001599, 63543006040, 63543006042,
       63543006048, 63543006051, 63543006052, 63543006650, 63543006054,
       63543006055, 63543007866, 63543007868, 63543013982, 63468005833,
       63468005834, 63468005837, 63468005839, 63468005841, 63468005843,
       63471005853, 63468005831, 63468005838, 63468005842, 63468005844,
       63513005961, 63513005729, 63513010758, 63513010760, 63513005954,
       63513007356, 63513005943, 63513005945, 63513009631, 63513005953,
       63513005959, 63513008461, 63513010759, 63513011990, 63513011880,
       63513012300, 63513005955, 63513013980, 64125009461, 60165010751,
       62637003935, 62637003947, 62637003961, 62637003978, 62637003980,
       64125006827, 64125006828, 64125006831, 64125013866]

### 4.2 Define Haversine formula Function<a id='4.2_Define_Haversine_formula_Function'></a>

In [16]:
# defining a  function to calculate distance between two locations 
# loc1= location of an houses
# loc2= location of schools
from haversine import Unit

def distance_from(loc1,loc2): 
    dist=hs.haversine(loc1,loc2,unit=Unit.MILES) # By default the haversine function returns distance in kms, so let's change it miles.
    return round(dist,2)

In [17]:
# running a loop which will parse houses location one by one to distance from function 
for _,row in school_loc.iterrows():
    house_loc[row.ncesId]=house_loc['coor'].apply(lambda x: distance_from(row.coor,x))

In [18]:
#Alright, now let's check haversine output: 
house_loc.head()

,address,lat,lon,coor,63513005942,63513005956,63513005958,60285000217,60285011276,60285000209,...,60165010751,62637003935,62637003947,62637003961,62637003978,62637003980,64125006827,64125006828,64125006831,64125013866
0,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,"(37.733978, -121.872736)",12.64,11.20,11.70,17.70,15.98,18.47,...,14.82,16.18,15.07,14.20,15.87,15.03,16.35,14.48,15.70,15.03
1,"270 birch creek dr, pleasanton, ca 94566",37.663585,-121.864563,"(37.663585, -121.86456299999999)",16.76,15.10,15.83,22.43,20.61,23.18,...,18.71,20.61,19.58,18.72,20.43,19.50,20.52,18.81,19.93,19.06
2,"5324 brookside ct, pleasanton, ca 94588",37.689374,-121.922526,"(37.689374, -121.922526)",13.66,11.89,12.75,21.34,19.77,22.15,...,15.42,17.70,16.76,15.93,17.66,16.65,17.39,15.83,16.85,15.85
3,"4385 clovewood ln, pleasanton, ca 94588",37.680132,-121.915110,"(37.6801319, -121.9151098)",14.41,12.64,13.50,21.84,20.23,22.65,...,16.18,18.44,17.49,16.66,18.39,17.39,18.14,16.57,17.61,16.60
4,"4273 trolan ln, dublin, ca 94568",37.737748,-121.876525,"(37.737747999999996, -121.8765253)",12.31,10.87,11.36,17.48,15.79,18.27,...,14.49,15.85,14.74,13.87,15.54,14.70,16.02,14.14,15.37,14.70


## 5. Haversine formula output Modification<a id='5_Haversine_formula_output_Modification'></a>

### 5.1 Melting the output<a id='5.1_Melting_the_output'></a>

In [20]:
#Now, let's reshape house_loc dataframe for better visibility using .melt(): house_school_loc
house_school_loc = pd.melt(house_loc, id_vars=['address', 'lat', 'lon'], value_vars=schools)
house_school_loc.head()

,address,lat,lon,variable,value
0,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,63513005942,12.64
1,"270 birch creek dr, pleasanton, ca 94566",37.663585,-121.864563,63513005942,16.76
2,"5324 brookside ct, pleasanton, ca 94588",37.689374,-121.922526,63513005942,13.66
3,"4385 clovewood ln, pleasanton, ca 94588",37.680132,-121.915110,63513005942,14.41
4,"4273 trolan ln, dublin, ca 94568",37.737748,-121.876525,63513005942,12.31


In [21]:
# let's check the shape of the data:
house_school_loc.shape

(1404468, 5)

**Alright, we have over 1.4 Milions rows now, so let's filter them out**

In [22]:
#Since haversine output gave us all schools for every single house.
#let's trim the value (distance house <> school) for 3 miles.
# So basically I want to get all schools around each house within 3 miles radius:
house_school_loc = house_school_loc[house_school_loc['value'] <=3]

In [23]:
# Again, let's check the shape of the data:
house_school_loc.shape

(86454, 5)

**Ok, 86K rows is much better :)**

In [24]:
# let's check out one single house to confirm:
house_school_loc[house_school_loc['address'] == '6985 calistoga ln, dublin, ca 94568']

,address,lat,lon,variable,value
390130,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,60001912078,1.26
408136,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,60001909275,2.99
420140,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,60001909278,2.92
426142,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,60001908443,1.76
432144,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,60001913169,1.32
438146,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,60001912927,1.71
450150,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,60001913862,1.88
834278,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,60002009286,2.86
888296,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,60002006951,2.95
1242414,"6985 calistoga ln, dublin, ca 94568",37.733978,-121.872736,63513010758,2.37


**let's now rename the columns so we can merge the dataframes later on**

In [25]:
# let's rename 'variable' to 'school' and 'value' to 'distance_miles':
house_school_loc.rename(columns = {list(house_school_loc)[3]: 'ncesId', list(house_school_loc)[4]: 'distance_miles'}, inplace = True)
house_school_loc.head()

,address,lat,lon,ncesId,distance_miles
608,"911 el pintado rd, danville, ca 94526",37.837165,-122.008754,63513005942,2.42
737,"387 cordell dr, danville, ca 94526",37.826980,-122.020212,63513005942,2.86
741,"309 garden creek pl, danville, ca 94526",37.829827,-122.006863,63513005942,2.91
815,"682 pixie ln, danville, ca 94526",37.826560,-122.015749,63513005942,2.95
826,"932 la gonda way, danville, ca 94526",37.837376,-122.014705,63513005942,2.26


**Now, lets get the school_gsrating dataframe so we can merge Haversine formula output with school_gsrating**

In [26]:
school_gsrating.head()

,gsId,name,type,gradeRange,enrollment,gsRating,parentRating,city,state,districtId,district,districtNCESId,address,phone,fax,website,ncesId,lat,lon
0,NaN,Bay Farm,public,K-8,610.0,9.0,5.0,Alameda,CA,1.0,Alameda Unified School District,601770.0,"200 Aughinbaugh Way, \nAlameda, CA 94502",(510) 748-4010,(510) 865-2194,http://bayfarm.alamedausd.ca.schoolloop.com,6.017700e+10,37.739000,-122.25195
1,NaN,Amelia Earhart Elementary School,public,K-5,651.0,10.0,5.0,Alameda,CA,1.0,Alameda Unified School District,601770.0,"400 Packet Landing Road, \nAlameda, CA 94502",(510) 748-4003,(510) 523-5837,http://earhart.alamedausd.ca.schoolloop.com,6.017700e+10,37.745290,-122.23928
2,NaN,Franklin Elementary School,public,K-5,335.0,8.0,5.0,Alameda,CA,1.0,Alameda Unified School District,601770.0,"1433 San Antonio Avenue, \nAlameda, CA 94501",(510) 748-4004,(510) 337-2439,http://franklin.alamedausd.ca.schoolloop.com,6.017700e+10,37.769035,-122.26160
3,NaN,Ruby Bridges Elementary School,public,K-5,484.0,3.0,5.0,Alameda,CA,1.0,Alameda Unified School District,601770.0,"351 Jack London Avenue, \nAlameda, CA 94501",(510) 748-4006,(510) 748-8314,http://rubybridges.alamedausd.ca.schoolloop.com/,6.017701e+10,37.781834,-122.28614
4,NaN,Maya Lin School,public,K-5,427.0,6.0,5.0,Alameda,CA,1.0,Alameda Unified School District,601770.0,"825 Taylor Street, \nAlameda, CA 94501",(510) 337-7007,(510) 523-8798,http://mls-alamedausd-ca.schoolloop.com,6.017701e+10,37.772724,-122.27189


In [27]:
#Now, let's trim school_gsrating by keeping only the columns we need: 'ncesId' and 'gsRating'
school_gsrating = school_gsrating[['ncesId', 'gsRating']]
school_gsrating.head()

,ncesId,gsRating
0,6.017700e+10,9.0
1,6.017700e+10,10.0
2,6.017700e+10,8.0
3,6.017701e+10,3.0
4,6.017701e+10,6.0


In [28]:
school_gsrating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ncesId    823 non-null    float64
 1   gsRating  708 non-null    float64
dtypes: float64(2)
memory usage: 13.0 KB


### 5.2 Merge the Dataframes<a id='5.2_Merge_the_Dataframes'></a>

In [29]:
# Finally, let's merge house_school_loc dataframe and school_gsrating:
house_school_loc_gsrating = pd.merge(house_school_loc, school_gsrating, how='left', on='ncesId')
house_school_loc_gsrating.head()

,address,lat,lon,ncesId,distance_miles,gsRating
0,"911 el pintado rd, danville, ca 94526",37.837165,-122.008754,63513005942,2.42,9.0
1,"387 cordell dr, danville, ca 94526",37.826980,-122.020212,63513005942,2.86,9.0
2,"309 garden creek pl, danville, ca 94526",37.829827,-122.006863,63513005942,2.91,9.0
3,"682 pixie ln, danville, ca 94526",37.826560,-122.015749,63513005942,2.95,9.0
4,"932 la gonda way, danville, ca 94526",37.837376,-122.014705,63513005942,2.26,9.0


In [30]:
house_school_loc_gsrating[house_school_loc_gsrating['address'] == '1 brisdale pl, antioch, ca 94509']

,address,lat,lon,ncesId,distance_miles,gsRating
899,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285000217,1.22,4.0
1559,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285000209,1.10,1.0
1925,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285000211,0.53,1.0
2969,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285000215,0.72,2.0
3306,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285009703,2.25,2.0
3713,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285000216,0.70,2.0
4338,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285011275,2.68,2.0
5167,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285000207,0.23,2.0
5482,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285000208,0.57,1.0
6421,"1 brisdale pl, antioch, ca 94509",38.002148,-121.819718,60285000212,1.08,2.0


### 5.3 Groupby the final dataframe<a id='5.3_Groupby_the_final_dataframe'></a>

In [31]:
# Alright, finally let's reshape the dataframe using groupby on address and taking the average of 'gsrating' & 'distance_miles':
final = house_school_loc_gsrating.groupby('address')['gsRating', 'distance_miles'].mean().reset_index()
final.head()

C:\Users\aktha\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,address,gsRating,distance_miles
0,"1 brisdale pl, antioch, ca 94509",2.000000,1.340000
1,"1 chorley walk, pleasant hill, ca 94523",6.272727,1.457273
2,"1 e circle ranch pl, san ramon, ca 94583",7.875000,1.703125
3,"1 hardy ct, danville, ca 94526",9.333333,1.621667
4,"1 heritage oaks rd, pleasant hill, ca 94523",6.384615,1.934615


## 6. Saving our final Dataframe<a id='6_Saving_our_final_Dataframe'></a>

In [ ]:
# Saving final:
final.to_csv(path_or_buf = "path/Feature_Engineering/datasets" + "/final.csv", index=False)

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))